In [220]:
import numpy as np
import time
from tic_env import TictactoeEnv, OptimalPlayer
env = TictactoeEnv()

# 2. Q-Learning

## 2.1 Learning from experts

In [214]:
def empty_pos(grid):
    '''return all empty positions of a grid'''
    avail = []
    for i in range(9):
        pos = (int(i/3), i % 3)
        if grid[pos] == 0:
            avail.append(pos)
    return avail

def randomMove(grid):
    """ Chose a random move from the available options. """
    avail = empty_pos(grid)
    return avail[np.random.randint(0, len(avail)-1)]


def find_Q_table_idx(Q_table, grid):
    """
    Checks if the state is already in the Q table. If not adds it and then returns the index
    """
    states = Q_table[:, 9]
    #print(states.shape)
    #print(states)
    #idx = np.where(states == state)
    #idx = np.where(np.array_equal(states, state))
    #print(idx)
    for idx, state in enumerate(states):
        #print(type(state))
        if np.array_equal(state, grid):
            return idx
        
        # If the state is not an array, then no state that corresponds to the grid was found
        elif isinstance(state, int):
            Q_table[idx, 9] = grid
            return idx

    raise ValueError('There was no more space in the Q-Table to add a new entry')
    

def pick_action(state, Q_table, epsilon=0):
    # implements greedy policy and returns the action with max. Q-value (given the state).
    # note: when Q-table is filled with zeros, returns a random policy. 
    if np.random.random() < epsilon:
        return randomMove(state)

    # Get index of Q_table corresponding to current state
    state_idx = find_Q_table_idx(Q_table, state)

    # Get all the valid actions converted to integers between 0 and 8
    valid_actions = [action_to_int(action) for action in empty_pos(state)]

    # Create a tuple with the corresponding actions and Q values
    action_scores = (valid_actions, Q_table[state_idx, :-1][valid_actions])

    # Find max Q value
    idxs_max_Q = np.where(action_scores[1] == np.max(action_scores[1]))[0]

    # Return random chosen action corresponding to max Q value
    return action_scores[0][np.random.choice(idxs_max_Q)]

def action_to_int(tuple):
    """
    Grids are of size 3x3. Hence to convert them to integers between 0 and 8, 
    we multiply the row idx by 3 and add the col idx
    """
    return 3*tuple[0] + tuple[1]

In [226]:
def q_learning(eps):
    players = ['X', 'O']
    Q_table = np.zeros((3**9, 10), dtype=object)
    #print(Q_table.shape)
    t_start = time.time()
    game_start = 0
    for game in range(20000):
        # Change starting player after every game
        players.reverse()
        env.reset()
        player_opt = OptimalPlayer(epsilon=0.5, player=players[0])
        end = False
        state, _, __ = env.observe()

        # for step in range(9):
        #     grid, _, __ = env.observe()
        #     if env.current_player == player_opt.player:
        #         move = player_opt.act(grid)
        #     else:
        #         move = pick_action(grid, Q_table, 0)
                
        #     grid_next, end, winner = env.step(move, print_grid=False)

        #     state_idx = find_Q_table_idx(Q_table, grid)
        #     state_next_idx = find_Q_table_idx(Q_table, grid_next)
        #     reward = env.reward(players[1])
        #     Q_table[state_idx, move] += eta * (reward + gamma * Q_table[state_next_idx, next_action] - Q_table[state_ind, action])

        #     if end:
        #         print('-------------------------------------------')
        #         print('Game end, winner is player ' + str(winner))
        #         print('Optimal player = ' +  players[0])
        #         print('Agent player = ' +  players[1])
        #         env.render()
        #         env.reset()
        #         break

        if env.current_player == player_opt.player:
            action = action_to_int(player_opt.act(state))
        else:
            action = pick_action(state, Q_table, eps)
    
        while not end:
            state_idx = find_Q_table_idx(Q_table, state)

            next_state, end, winner = env.step(action)
            state_next_idx = find_Q_table_idx(Q_table, next_state)

            if not end:
                if env.current_player == player_opt.player:
                    next_action = action_to_int(player_opt.act(next_state))
                else:
                    next_action = pick_action(next_state, Q_table, 0)

            # update Q-table using the iterative update rule   
            reward = env.reward(players[1])      
            Q_table[state_idx, action] += 0.5 * (reward + 0.99 * Q_table[state_next_idx, next_action] - Q_table[state_idx, action])

            state = next_state
            action = next_action
        
        if(game>19980 or game < 4):
            print('-------------------------------------------')
            print('Game {} ended, winner is player {}'.format(game, str(winner)))
            print('Optimal player = ' +  players[0])
            print('Agent player = ' +  players[1])
            env.render()

        if game != 0 and game%1000 == 0:
            t_end = time.time()
            print('Time for games {}-{}: {:.2f}s <-> {:.2f}m'.format(game_start, game,(t_end - t_start), (t_end - t_start)/60))
            t_start = time.time()
            game_start = game

    return Q_table.copy()


In [227]:
generted_Q_table = q_learning(0)


-------------------------------------------
Game 0 ended, winner is player None
Optimal player = O
Agent player = X
|X O O|
|O X X|
|X X O|

-------------------------------------------
Game 1 ended, winner is player O
Optimal player = X
Agent player = O
|O X X|
|O X X|
|O O -|

-------------------------------------------
Game 2 ended, winner is player O
Optimal player = O
Agent player = X
|O X X|
|- O -|
|- X O|

-------------------------------------------
Game 3 ended, winner is player X
Optimal player = X
Agent player = O
|- - X|
|- X O|
|X - O|



KeyboardInterrupt: 

In [183]:
grid = np.zeros((3,3))
grid_ = grid.copy()
grid_[0,0] = 1
grid_[1,0] = -1
grid_[0,1] = 1
grid_[0,2] = -1

print(grid_[0,:])
print(grid_[0, :-1])

q = np.zeros((5, 10), dtype=object)
#print(grid)
print(q.shape)
print(q)
q[0, 0] = grid_
q[1, 0] = grid
q[2, 0] = grid
print(q)
q[0, 1:] = [1,2,3,41,5,61,7,61,9]
q[1, 1:] = [11,12,13,14,15,16,17,18,19]
q[2, 1:] = [21,22,23,24,25,26,27,28,28]
print(q[0,:])
print(q.shape)

valid_actions = np.where(q[0, 0]==0)
print(list(zip(valid_actions[0], valid_actions[1])))

idx_valid_actions = [3*x + y for x, y in zip(valid_actions[0], valid_actions[1])]
print(idx_valid_actions)

action_scores_tup = (idx_valid_actions, q[0, 1:][idx_valid_actions])
print(action_scores_tup)
idx_max_action = np.where(action_scores_tup[1] == np.max(action_scores_tup[1]))[0]
print(idx_max_action)
random_idx = np.random.choice(idx_max_action)
print(random_idx)
print(action_scores_tup[0][random_idx])


def update(q):
    q[0,0] = grid

update(q)
print(q)

idx = find_Q_table_idx(q, np.ones((3,3)))
print(idx)
print(q)

[ 1.  1. -1.]
[1. 1.]
(5, 10)
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[array([[ 1.,  1., -1.],
         [-1.,  0.,  0.],
         [ 0.,  0.,  0.]]) 0 0 0 0 0 0 0 0 0]
 [array([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]) 0 0 0 0 0 0 0 0 0]
 [array([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]) 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[array([[ 1.,  1., -1.],
        [-1.,  0.,  0.],
        [ 0.,  0.,  0.]]) 1 2 3 41 5 61 7 61 9]
(5, 10)
[(1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
[4, 5, 6, 7, 8]
([4, 5, 6, 7, 8], array([5, 61, 7, 61, 9], dtype=object))
[1 3]
1
5
[[array([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]) 1 2 3 41 5 61 7 61 9]
 [array([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]) 11 12 13 14 15 16 17 18 19]
 [array([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]) 21 22 23 24 25 26 27 28 28]
 [0 0 0 0 0 